In [2]:
import torch
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add to sys.path if not already there
if project_root not in sys.path:
    sys.path.append(project_root)

from model_utils import BasicCNN
from data_prep.data_utils import ClassificationDataset, default_transforms

torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [3]:
model = BasicCNN()
train_data = ClassificationDataset('../data/train', transform=default_transforms)
val_data = ClassificationDataset('../data/valid', transform=default_transforms)
model.fit(train_loader=train_data.get_dataloader(), epochs=10)

[1,  2000] loss: 1.701
[2,  2000] loss: 1.507
[3,  2000] loss: 1.438
[4,  2000] loss: 1.394
[5,  2000] loss: 1.359
[6,  2000] loss: 1.346
[7,  2000] loss: 1.327
[8,  2000] loss: 1.307
[9,  2000] loss: 1.293
[10,  2000] loss: 1.279


In [ ]:
test_data = ClassificationDataset('../data/test', transform=default_transforms)
correct_pred, total_pred = model.evaluate(test_data.get_dataloader())